In [1]:
import cpa
import scanpy as sc

[rank: 0] Global seed set to 0
2024-06-13 14:25:30,573	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-06-13 14:25:35,021	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [2]:
import gdown

gdown.download("https://drive.google.com/uc?export=download&id=1RRV0_qYKGTvD3oCklKfoZQFYqKJy4l6t")
data_path = "combo_sciplex_prep_hvg_filtered.h5ad"
adata = sc.read(data_path)

Downloading...
From (original): https://drive.google.com/uc?export=download&id=1RRV0_qYKGTvD3oCklKfoZQFYqKJy4l6t
From (redirected): https://drive.google.com/uc?export=download&id=1RRV0_qYKGTvD3oCklKfoZQFYqKJy4l6t&confirm=t&uuid=b913c87b-7c21-4ef5-9a9a-00d160d56ac5
To: /ictstr01/home/icb/dominik.klein/git_repos/ot_pert_new/competing_methods/combosciplex/CPA/combo_sciplex_prep_hvg_filtered.h5ad
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 490M/490M [00:04<00:00, 115MB/s]


In [3]:
adata_train_path = "/lustre/groups/ml01/workspace/ot_perturbation/data/combosciplex/adata_train_30.h5ad"
adata_test_path = "/lustre/groups/ml01/workspace/ot_perturbation/data/combosciplex/adata_test_30.h5ad"
adata_ood_path = "/lustre/groups/ml01/workspace/ot_perturbation/data/combosciplex/adata_ood_30.h5ad"

In [4]:
adata_train = sc.read(adata_train_path)
adata_test = sc.read(adata_test_path)
adata_ood = sc.read(adata_ood_path)

In [5]:
adata_train.obs["split"].value_counts()

split
train    51882
Name: count, dtype: int64

In [6]:
adata_test.obs["split"].value_counts()

split
test    3100
Name: count, dtype: int64

In [7]:
adata_ood.obs["split"].value_counts()

split
ood    8896
Name: count, dtype: int64

In [8]:
import anndata

adata_combined = anndata.concat((adata_train, adata_test, adata_ood), label="split", keys=["train", "test", "ood"])

In [9]:
adata.obs["index"] = adata.obs_names.values
adata.obs["index_adapted"] = adata.obs.apply(lambda x: x["index"][:-2], axis=1)
adata.obs.set_index("index_adapted", inplace=True)

In [10]:
adata_combined.obs["cov_drug_dose"] = adata.obs["cov_drug_dose"]
adata_combined.obs["condition_ID"] = adata.obs["condition_ID"]
adata_combined.obs["log_dose"] = adata.obs["log_dose"]

In [11]:
frac_valid = adata[adata.obs["split_1ct_MEC"] == "valid"].n_obs / (
    adata[adata.obs["split_1ct_MEC"] == "valid"].n_obs + adata[adata.obs["split_1ct_MEC"] == "train"].n_obs
)

In [12]:
import numpy as np


def create_split(x):
    if x["split"] != "train":
        return x["split"]
    is_train = np.random.choice(2, p=[frac_valid, 1 - frac_valid])
    if is_train:
        return "train_train"
    return "train_valid"


adata_combined.obs["split_1ct_MEC"] = adata_combined.obs.apply(create_split, axis=1)

In [13]:
adata_combined.obs["split_1ct_MEC"].value_counts()

split_1ct_MEC
train_train    46592
ood             8896
train_valid     5290
test            3100
Name: count, dtype: int64

In [14]:
adata_combined.obs["condition_ID"] = adata_combined.obs["condition_ID"].cat.remove_unused_categories()

In [15]:
cpa.CPA.setup_anndata(
    adata_combined,
    perturbation_key="condition_ID",
    dosage_key="log_dose",
    control_group="CHEMBL504",
    batch_key=None,
    is_count_data=True,
    categorical_covariate_keys=["cell_type"],
    # deg_uns_key='rank_genes_groups_cov',
    deg_uns_cat_key="cov_drug_dose",
    max_comb_len=2,
)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 63878/63878 [00:00<00:00, 966671.90it/s]
An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.


INFO     Generating sequential column names                                                                        
INFO     Generating sequential column names                                                                        


In [16]:
ae_hparams = {
    "n_latent": 128,
    "recon_loss": "gauss",
    "doser_type": "logsigm",
    "n_hidden_encoder": 512,
    "n_layers_encoder": 3,
    "n_hidden_decoder": 512,
    "n_layers_decoder": 3,
    "use_batch_norm_encoder": True,
    "use_layer_norm_encoder": False,
    "use_batch_norm_decoder": True,
    "use_layer_norm_decoder": False,
    "dropout_rate_encoder": 0.1,
    "dropout_rate_decoder": 0.1,
    "variational": False,
    "seed": 434,
}

trainer_params = {
    "n_epochs_kl_warmup": None,
    "n_epochs_pretrain_ae": 30,
    "n_epochs_adv_warmup": 50,
    "n_epochs_mixup_warmup": 3,
    "mixup_alpha": 0.1,
    "adv_steps": 2,
    "n_hidden_adv": 64,
    "n_layers_adv": 2,
    "use_batch_norm_adv": True,
    "use_layer_norm_adv": False,
    "dropout_rate_adv": 0.3,
    "reg_adv": 20.0,
    "pen_adv": 20.0,
    "lr": 0.0003,
    "wd": 4e-07,
    "adv_lr": 0.0003,
    "adv_wd": 4e-07,
    "adv_loss": "cce",
    "doser_lr": 0.0003,
    "doser_wd": 4e-07,
    "do_clip_grad": False,
    "gradient_clip_value": 1.0,
    "step_size_lr": 45,
}

In [17]:
model = cpa.CPA(
    adata=adata_combined,
    split_key="split_1ct_MEC",
    train_split="train_train",
    valid_split="train_valid",
    **ae_hparams,
)

[rank: 0] Global seed set to 434


In [18]:
model.train(
    max_epochs=200,
    use_gpu=True,
    batch_size=128,
    plan_kwargs=trainer_params,
    early_stopping_patience=10,
    check_val_every_n_epoch=5,
    save_path="/lustre/groups/ml01/workspace/ot_perturbation/models/cpa/combosciplex",
)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████| 32/32 [00:00<00:00, 94.08it/s]
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Epoch 5/200:   2%|▌                         | 4/200 [01:28<1:08:57, 21.11s/it, v_num=1, recon=-2.62e+3, r2_mean=0.782, adv_loss=2.5, acc_pert=0.147]


Epoch 00004: cpa_metric reached. Module best state updated.


Epoch 10/200:   4%| | 9/200 [03:16<1:07:20, 21.15s/it, v_num=1, recon=-3.05e+3, r2_mean=0.788, adv_loss=2.36, acc_pert=0.172, val_recon=-2.79e+3, di


Epoch 00009: cpa_metric reached. Module best state updated.



disnt_basal = 0.06287326923523434
disnt_after = 0.07675864522399398
val_r2_mean = 0.7876624396081424
val_r2_var = 0.4331447526952615
Epoch 15/200:   7%| | 14/200 [05:03<1:05:26, 21.11s/it, v_num=1, recon=-3.27e+3, r2_mean=0.792, adv_loss=2.29, acc_pert=0.195, val_recon=-3.05e+3, d


Epoch 00014: cpa_metric reached. Module best state updated.


Epoch 20/200:  10%| | 19/200 [06:50<1:03:48, 21.15s/it, v_num=1, recon=-3.42e+3, r2_mean=0.792, adv_loss=2.28, acc_pert=0.195, val_recon=-3.23e+3, d


Epoch 00019: cpa_metric reached. Module best state updated.



disnt_basal = 0.05173956598467181
disnt_after = 0.0651405600504188
val_r2_mean = 0.7938699751023748
val_r2_var = 0.4358818104586815
Epoch 30/200:  14%|▏| 29/200 [10:23<59:56, 21.03s/it, v_num=1, recon=-3.63e+3, r2_mean=0.795, adv_loss=2.26, acc_pert=0.206, val_recon=-3.41e+3, dis
disnt_basal = 0.049802296656969135
disnt_after = 0.05998603401444358
val_r2_mean = 0.7947056554491392
val_r2_var = 0.43485590499608356
Epoch 40/200:  20%|▏| 39/200 [13:49<54:10, 20.19s/it, v_num=1, recon=-3.72e+3, r2_mean=0.797, adv_loss=2.33, acc_pert=0.196, val_recon=-3.49e+3, dis
disnt_basal = 0.04811781969603754
disnt_after = 0.057403161715907605
val_r2_mean = 0.7922981460784866
val_r2_var = 0.4298887801895697
Epoch 50/200:  24%|▏| 49/200 [17:12<49:57, 19.85s/it, v_num=1, recon=-3.77e+3, r2_mean=0.796, adv_loss=2.46, acc_pert=0.176, val_recon=-3.53e+3, dis
disnt_basal = 0.04738994364516257
disnt_after = 0.05602701979232046
val_r2_mean = 0.7944286830793234
val_r2_var = 0.4299598283702646
Epoch 60/200:  30%

In [19]:
ctrl_adata = adata_ood[adata_ood.obs['condition'] == 'control'].copy()

In [20]:
adata_combined.layers["X_true"] = adata_combined.X
adata_combined.X = ctrl_adata.X[np.random.choice(ctrl_adata.n_obs, size=adata_combined.n_obs, replace=True), :]

In [21]:
model.predict(adata_combined)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 1997/1997 [00:08<00:00, 244.97it/s]


In [22]:
adata_combined.X = adata_combined.layers["X_true"]

In [23]:
adata_combined.write(
    "/lustre/groups/ml01/workspace/ot_perturbation/models/cpa/combosciplex/adata_with_predictions_30.h5ad"
)